In [ ]:
from IPython.display import Code

Code(filename='cluster.tf', language='terraform')

In [ ]:
!terraform apply -auto-approve

In [ ]:
from IPython.display import Code

Code(filename='kubeconfig.yaml', language='yaml')

In [ ]:
!kubectl config view

In [ ]:
!kubectl apply -f https://github.com/parca-dev/parca/releases/download/v0.22.0/kubernetes-manifest.yaml

In [ ]:
!kubectl apply -f https://github.com/parca-dev/parca-agent/releases/download/v0.33.1/kubernetes-manifest.yaml

In [ ]:
!kubectl get pods -n parca -o wide -w

In [ ]:
import os
get_ipython().system = os.system
!kubectl -n parca port-forward service/parca 7070 & echo $! > /tmp/parca-forward.pid

In [ ]:
!kill $(cat /tmp/parca-forward.pid)

In [ ]:
from IPython.display import display, HTML

display(HTML('<a href="http://localhost:7070" target="_blank">Open Parca UI</a>'))

In [ ]:
!cd demo; go build -o demo .

In [ ]:
Code(filename='demo/Dockerfile', language='Docker')

In [ ]:
!docker build --platform linux/amd64 -t ghcr.io/guilhem/pgo-demo:nopgo -f demo/Dockerfile --push demo/

In [ ]:
!kubectl run pgo-demo --image=ghcr.io/guilhem/pgo-demo:nopgo --port=8080

In [ ]:
import os
get_ipython().system = os.system
!kubectl port-forward pods/pgo-demo 8080 & echo $! > port-forward.pid

In [ ]:
!kill $(cat port-forward.pid)

In [ ]:
!curl -o test.md -L "https://raw.githubusercontent.com/golang/go/c16c2c49e2fa98ae551fc6335215fadd62d33542/README.md"

In [ ]:
!curl --data-binary @test.md http://localhost:8080/render

In [ ]:
!go run github.com/prattmic/markdown-pgo/load@latest

In [ ]:
from IPython.display import display, HTML

url = "http://localhost:7070/?query_browser_mode=advanced&expression_a=parca_agent%3Asamples%3Acount%3Acpu%3Ananoseconds%3Adelta%7Bnamespace%3D%22default%22%2C%20run%3D%22pgo-demo%22%7D&from_a=1729002421615&to_a=1729003321615&time_selection_a=relative:minute|15&sum_by_a=__none__&merge_from_a=1729002421615&merge_to_a=1729003321615&selection_a=parca_agent%3Asamples%3Acount%3Acpu%3Ananoseconds%3Adelta%7Bnamespace%3D%22default%22%2C%20run%3D%22pgo-demo%22%7D"
display(HTML(f'<a href="{url}" target="_blank">Open Parca Query</a>'))

In [3]:
%%bash
cd demo/
mv ~/Downloads/profile.pb.gz ./profile.pb.gz
gunzip -f profile.pb.gz

In [ ]:
!cd demo; go build -o demo-pgo -pgo ./profile.pb .

In [ ]:
from IPython.display import Code

Code(filename='demo/Dockerfile.pgo', language='Docker')

In [ ]:
!docker build --platform linux/amd64 -t ghcr.io/guilhem/pgo-demo:pgo -f demo/Dockerfile.pgo --push demo/

In [ ]:
!kubectl run pgo-demo-pgo --image=ghcr.io/guilhem/pgo-demo:pgo --port=8080

In [ ]:
import os
get_ipython().system = os.system
!kubectl port-forward pods/pgo-demo-pgo 8081:8080 & echo $! > port-forward-pgo.pid

In [1]:
!kill $(cat port-forward-pgo.pid)

In [ ]:
!go get github.com/prattmic/markdown-pgo/load@latest

In [ ]:
! go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr http://localhost:8080 > nopgo.txt
from IPython.display import Code
Code(filename='nopgo.txt')

In [ ]:
!go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr "http://localhost:8081" > withpgo.txt

from IPython.display import Code
Code(filename='withpgo.txt')

In [ ]:
!go install golang.org/x/perf/cmd/benchstat@latest

In [ ]:
%%bash
benchstat nopgo.txt withpgo.txt

In [ ]:
!terraform destroy -auto-approve

In [ ]:
# run ./demo/demo
! go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr http://localhost:8080 > nopgo-local.txt

In [ ]:
# run ./demo/demo-pgo
! go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr http://localhost:8080 > withpgo-local.txt

In [ ]:
%%bash
benchstat nopgo-local.txt withpgo-local.txt